In [1]:
isOpenset = False
existSavedFile = True

isLocalServer = True
sys_path_to_be_added = '/notebook/personal/ksuchoi216/FaceID_model_rebuild/'
config_path = './configs/config.json'

%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import os
import numpy as np

def checkLocalServer(isLocalServer, sys_path_to_be_added=None):
    print(f'isLocalServer is {isLocalServer}')
    if isLocalServer is True:
        if sys_path_to_be_added not in sys.path:
            sys.path.insert(0, sys_path_to_be_added)
            os.chdir(sys_path_to_be_added)

checkLocalServer(isLocalServer, sys_path_to_be_added)

from utils import Config_Manager
cfgm = Config_Manager(config_path)
cfg = cfgm.get_cfg()
!pwd
!nvidia-smi

isLocalServer is True
/notebook/personal/ksuchoi216/FaceID_model_rebuild
Fri Oct 14 17:08:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:04:00.0 Off |                    0 |
| N/A   34C    P0    33W / 250W |   2392MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P1

In [6]:
from data_preprocessing import FolderDataset, DatasetFromNumpy
from data_preprocessing import buildDataLoaders, saveDataloaders

face_threshold = 0.97
cfg["FolderDataset"]["folder_for_raw"] = cfg["folder_for_raw"]
folderDataset = FolderDataset(cfg["FolderDataset"], face_threshold)
save_folder = 'extracted_np_'+str(int(face_threshold*100))
print(f'save_folder: {save_folder}')

folderDataset.setFilePath(
    './data',
    save_folder,
    'face',
    '.npy'
)

if not existSavedFile:
    print("loading numpy datasets...")
    emb, lb = folderDataset.createNumpyData()
    folderDataset.saveToNumpy(emb, lb)
emb, lb = folderDataset.loadFromNumpy()
print("creating a dataset from numpy datasets")



face_threshold: 0.97
device is cuda:0
image size: 140
Loading faces from ./raw/photos_from_video
0 jhoh
1 jhongyoo
2 jhyoo
3 kschoi
save_folder: extracted_np_97
path_emb: ./data/extracted_np_97/face_emb.npy
path_lb: ./data/extracted_np_97/face_lb.npy
loaded emb:(1157, 512) lb:(1157,)
creating a dataset from numpy datasets


In [13]:
from sklearn.model_selection import train_test_split

def splitNumpy(
    data: np.ndarray,
    labels: np.ndarray,
    ratios: dict,
    exist_val=True):
    
    res = {}
    
    if not exist_val and (ratios['train']+ratios['val']) != 1:
        return print('fail ratios')
    
    X_train, X_test, y_train, y_test = train_test_split(
        data,
        labels,
        test_size=(1-ratios['train'])
    )
    res['train'] = [X_train, y_train]
    
    if exist_val:
        X_val, X_test, y_val, y_test = train_test_split(
            X_test,
            y_test,
            test_size=ratios['test']
        )
        res['val'] = [X_val, y_val]
        res['test'] = [X_test, y_test]
    else:
        res['test'] = [X_test, y_test]
        
    return res


    
ratios = {'train': 0.7, 'val': 0.2, 'test': 0.1}
print(ratios.values())

res = splitNumpy(data=emb, labels=lb, ratios=ratios, exist_val=True)

dict_values([0.7, 0.2, 0.1])
{'train': [array([[ 0.0509004 , -0.02280433,  0.02979773, ...,  0.014034  ,
         0.01821957, -0.00122135],
       [ 0.05406609,  0.01823001,  0.01457361, ..., -0.0319244 ,
         0.04040344,  0.04988686],
       [ 0.07134515,  0.0364401 ,  0.01726693, ..., -0.02950949,
         0.03904522,  0.07198531],
       ...,
       [ 0.05788729, -0.02598635, -0.0287576 , ..., -0.06271928,
         0.02294333, -0.0203035 ],
       [ 0.05158824, -0.02065706, -0.04635647, ..., -0.05895969,
         0.0220195 , -0.03833106],
       [ 0.05388598,  0.02383908,  0.01256889, ..., -0.04189012,
         0.02393685,  0.07472246]], dtype=float32), array([1, 0, 0, 0, 0, 3, 0, 1, 1, 1, 2, 3, 3, 2, 2, 3, 1, 2, 0, 0, 0, 3,
       3, 1, 3, 0, 1, 3, 3, 0, 3, 3, 2, 2, 3, 2, 3, 0, 1, 2, 0, 1, 2, 3,
       2, 1, 2, 1, 0, 0, 1, 0, 2, 0, 1, 3, 3, 0, 3, 2, 0, 3, 3, 3, 3, 2,
       1, 3, 1, 1, 1, 0, 3, 1, 0, 3, 1, 0, 3, 2, 0, 3, 0, 2, 3, 0, 0, 0,
       0, 2, 1, 0, 2, 2, 3, 3, 1, 3, 0,

In [38]:
from data_preprocessing import filterNumpy

if isOpenset:
    kkc = [0, 1, 2]
    kuc = [3]
    
    emb_kkc, lb_kkc = filterNumpy(
        data = emb,
        labels = lb,
        selected_labels = kkc
    )
    
    emb_kuc, emb_kuc = filterNumpy(
        data = emb,
        labels = lb,
        selected_labels = kuc
    )



    
 

ImportError: cannot import name 'filterNumpy' from 'data_preprocessing' (/notebook/personal/ksuchoi216/FaceID_model_rebuild/data_preprocessing/__init__.py)

In [ ]:
   
dataset = DatasetFromNumpy(emb_, lb_)
dataloaders = buildDataLoaders(dataset, 4, 0.7, 0.2)
print(f'save_folder: {save_folder}')
saveDataloaders(source_folder='data',
                save_folder='dataloader_org',
                dataloaders=dataloaders)

In [4]:
# from data_preprocessing import FolderDataset, DatasetFromNumpy
# from data_preprocessing import buildDataLoaders, saveDataloaders

# face_thresholds = []

# for face_threshold in face_thresholds:
#     cfg["FolderDataset"]["folder_for_source"] = cfg["folder_for_source"]
#     folderDataset = FolderDataset(cfg["FolderDataset"], face_threshold)
#     save_folder = 'extracted_np_'+str(int(face_threshold*100))
#     print(f'save_folder: {save_folder}')
    
#     folderDataset.setFilePath(
#         './data',
#         save_folder,
#         'face',
#         '.npy'
#     )

#     if isUnloadable:
#         print("loading numpy datasets...")
#         np_emb, np_lb = folderDataset.createNumpyData()
#         folderDataset.saveToNumpy(np_emb, np_lb)
#     np_emb, np_lb = folderDataset.loadFromNumpy()
#     print("creating a dataset from numpy datasets")
#     dataset = DatasetFromNumpy(np_emb, np_lb)
    
    
#     if isDataloader:
#         dataloaders = buildDataLoaders(dataset, 4, 0.7, 0.2)
#         save_folder = 'dataloader_'+str(int(threshold*100))
#         print(f'save_folder: {save_folder}')
#         saveDataloaders(source_folder='data',
#                         save_folder=save_folder,
#                         dataloaders=dataloaders)


threshold: 0.6
device is cuda:0
image size: 240
Loading faces from ./source/faces_from_photos
0 jhoh
1 jhongyoo
2 jhyoo
3 kschoi
data length: 400
save_folder: extracted_np_60
path_emb: ./data/extracted_np_60/face_emb.npy
path_lb: ./data/extracted_np_60/face_lb.npy
loading numpy datasets...
[   0]converting to numpy...
[   1]converting to numpy...
[   2]converting to numpy...
[   3]converting to numpy...
[   4]converting to numpy...
[   5]converting to numpy...
[   6]converting to numpy...
[   7]converting to numpy...
[   8]converting to numpy...
[   9]converting to numpy...
[  10]converting to numpy...
[  11]converting to numpy...
[  12]converting to numpy...
[  13]converting to numpy...
[  14]converting to numpy...
[  15]converting to numpy...
[  16]converting to numpy...
[  17]converting to numpy...
[  18]converting to numpy...
[  19]converting to numpy...
[  20]converting to numpy...
[  21]converting to numpy...
[  22]converting to numpy...
[  23]converting to numpy...
[  24]convert